In [1]:
from ultralytics import YOLO
import os, glob

model = YOLO("runs/detect/train/weights/best.pt")
input_dir = "raw_images"
output_dir = "cropped_billboards"
os.makedirs(output_dir, exist_ok=True)

for img_path in glob.glob(f"{input_dir}/*.jpg"):
    results = model(img_path)
    for r in results:
        r.save_crop(output_dir) # Save cropped images to the output directory   


image 1/1 d:\img_processing\raw_images\006a2bc2-4b0c-4d6f-8d7f-c4da51b1c826.jpg: 640x480 1 mupi, 72.3ms
Speed: 2.4ms preprocess, 72.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 d:\img_processing\raw_images\015ce3e6-4f63-4547-8e06-3b3ca5896d8a.jpg: 640x480 1 mupi, 57.2ms
Speed: 1.9ms preprocess, 57.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 d:\img_processing\raw_images\0188702c-e5c8-401c-8850-41c8024027eb.jpg: 640x480 1 mupi, 55.9ms
Speed: 1.7ms preprocess, 55.9ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 d:\img_processing\raw_images\01a54d78-b747-41b7-ba7d-50eb66322c10.jpg: 640x480 1 mupi, 55.6ms
Speed: 1.5ms preprocess, 55.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 d:\img_processing\raw_images\01a659df-2e94-4c7a-a6c8-ab797e2954d4.jpg: 480x640 1 rectangle-wide-board, 63.1ms
Speed: 1.6ms preprocess, 63.1ms inference, 0.8ms postprocess per image 

In [2]:
import cv2
def preprocess(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    denoised = cv2.bilateralFilter(gray, 9, 75, 75)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)).apply(denoised)
    return clahe

In [3]:
from ultralytics import YOLO
import cv2
import os
import pytesseract
from PIL import Image
from pathlib import Path

# Path to trained YOLOv8 model
model_path = "runs/detect/train/weights/best.pt"
model = YOLO(model_path)

# Folder with raw input images
input_dir = "raw_images"
output_dir = "cropped_billboards"
os.makedirs(output_dir, exist_ok=True)

In [4]:
def crop_and_save_billboards(image_path, model, output_folder):
    results = model(image_path)
    image = cv2.imread(image_path)
    basename = Path(image_path).stem

    cropped_paths = []

    for i, r in enumerate(results):
        boxes = r.boxes.xyxy.cpu().numpy().astype(int)
        for j, (x1, y1, x2, y2) in enumerate(boxes):
            cropped = image[y1:y2, x1:x2]
            crop_path = os.path.join(output_folder, f"{basename}_{j}.jpg")
            cv2.imwrite(crop_path, cropped)
            cropped_paths.append(crop_path)
    
    return cropped_paths

In [ ]:
import json
from tqdm import tqdm

all_data = []

for img_file in tqdm(list(Path(input_dir).glob("*.jpg"))):
    cropped_imgs = crop_and_save_billboards(str(img_file), model, output_dir)
    for crop_img in cropped_imgs:
        text = extract_text(crop_img)
        all_data.append({
            "original_image": img_file.name,
            "cropped_image": Path(crop_img).name,
            "text": text
        })

# Save to JSON
with open("billboard_text_data.json", "w", encoding="utf-8") as f:
    json.dump(all_data, f, indent=2, ensure_ascii=False)


  0%|          | 0/470 [00:00<?, ?it/s]


image 1/1 d:\img_processing\raw_images\006a2bc2-4b0c-4d6f-8d7f-c4da51b1c826.jpg: 640x480 1 mupi, 60.6ms
Speed: 2.1ms preprocess, 60.6ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 480)


  0%|          | 0/470 [00:00<?, ?it/s]


TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.